In [1]:
import sys, win32com.client
import pandas as pd
import numpy as np
import datetime
import sapscript as ss

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None

In [2]:
def replace_minus(string):
    string = string.replace('.','')
    string = string.replace(' ','')
    if string[-1] == '-':
        string = -int(string.replace('-',''))
    else:
        string = int(string)
    return(string)




def read_zam(session, group_number):
    """Читаем группу замещающих товаров"""
    zam = []
    #заходим в группы замещ.товаров с номером group_number
    session.StartTransaction(Transaction="zgrzt3")
    session.findById("wnd[0]/usr/ctxtS_DPGID-LOW").text = group_number
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    
    #в верхней ALV проваливаемся в группу
    grid1 = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[0]/shell")
    grid1.clickCurrentCell()
    
    #в нижней ALV копируем товары и их приоритеты в отдельный датафрейм
    grid2 = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[1]/shell")
    
    cols = ['MATNR','DPGPR','MSTAE_' + company_code]
    cols_name = ['Тов','Пр','ЖЦ']
    
    dataframe = ss.read_alv(grid2, cols, cols_name)
    dataframe['Пр'] = dataframe['Пр'].astype('int32')
    
    return dataframe





def read_stock(session, plants, materials):
    """Читаем запасы на 0010 складе в ZMB52"""
    session.StartTransaction(Transaction="zmb52")
    session.findById("wnd[0]/usr/btn%_SO_WERKS_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    #копируем заводы и товары в буфер
    werks = pd.DataFrame(plants)
    werks.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/usr/ctxtSO_LGORT-LOW").text = "0010"

    session.findById("wnd[0]/usr/btn%_SO_MATNR_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    
    #копируем товары и товары в буфер
    materials.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    
    #Ставим флаги резервов
    session.findById("wnd[0]/usr/chkP_REZERV").selected = 0
    session.findById("wnd[0]/usr/chkP_IMDCQ").selected = 0
    
    session.findById("wnd[0]/usr/ctxtP_VAR").text = variant_zmb52
    
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    
    #формат вывода
    session.findById("wnd[0]/shellcont/shell").pressToolbarButton("&MB_VARIANT")
    session.findById("wnd[1]/tbar[0]/btn[71]").press()
    session.findById("wnd[2]/usr/txtGS_SEARCH-VALUE").text = variant_zmb52
    session.findById("wnd[2]/usr/txtGS_SEARCH-VALUE").caretPosition = 7
    session.findById("wnd[2]/tbar[0]/btn[0]").press()
    session.findById("wnd[2]/tbar[0]/btn[12]").press()
    session.findById("wnd[1]/usr/cntlGRID/shellcont/shell").clickCurrentCell()

    
    grid = session.findById("wnd[0]/shellcont/shell")
    
    cols = ['WERKS','MATNR','QTY']
    cols_name = ['Звд','Тов','Запас']
    
    dataframe = ss.read_alv(grid, cols, cols_name)
    #Обрабатываем минусы и пробелы у запасов, преобразуя их в int
    dataframe['Запас'] = dataframe['Запас'].apply(lambda x: ss.replace_minus_int(x))
    
    return dataframe





def count_inact_stock(zam, stock):
    """Считаем количество неактивных товаров с запасом на магазине"""
    #Размножаем все замещающие товары по количеству заводов и добавляем к ним запасы.
    zam_with_plant = pd.DataFrame()
    for plant in plants:
        zam['Звд'] = plant
        zam_with_plant = zam_with_plant.append(zam)
    zam = zam.drop(columns='Звд')
    zam_with_stock = zam_with_plant.merge(stock, on=['Тов','Звд'], how='outer')
    zam_with_stock.fillna(0, inplace=True)
    zam_with_stock['Запас'] = zam_with_stock['Запас'].astype(int)
    cols = ['Звд', 'Тов','ЖЦ','Пр','Запас']
    zam_with_stock = zam_with_stock[cols]
    
    #Находим количество неактивных товаров с запасом на каждом из заводов
    #пробуем 2 варианта: с учетом сегментов ЖЦи без учета
    #inact_stock = zam_with_stock[(zam_with_stock['ЖЦ'].isin(inactive_segment)) & (zam_with_stock['Запас']>0)].groupby('Звд').count()['Запас']
    inact_stock = zam_with_stock[(zam_with_stock['ЖЦ'].isin(inactive_segment)) & (zam_with_stock['Запас']>0)].groupby('Звд').count()['Запас']
    #Если вдруг попались заводы без запасов вообще, то добавляем их в список с нулем
    for plant in plants:
        if plant not in inact_stock.index:
            inact_stock = inact_stock.append(pd.Series([0],index = [plant]))
    return inact_stock



def read_az(session, materials):
    #заходим в позиции проектов
    session.StartTransaction(Transaction="zreplen03")
    session.findById("wnd[0]/usr/ctxtP_DATE").text = date
    session.findById("wnd[0]/usr/ctxtS_DATA0-LOW").text = ""
    
    #Вставляем заводы из буфера
    session.findById("wnd[0]/usr/btn%_S_EMWRK_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    #копируем заводы в буфер
    werks = pd.DataFrame(plants)
    werks.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    
    #Вставляем список товаров из буфера
    session.findById("wnd[0]/usr/btn%_S_MATNR_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    #копируем товары в буфер
    materials.to_clipboard(index=False, header=False, line_terminator='\r\n')
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    
    session.findById("wnd[0]/usr/ctxtS_SDWRK-LOW").text = vendor
    session.findById("wnd[0]/usr/radVIEW_POS").select()
    session.findById("wnd[0]/usr/ctxtP_VARI").text = variant
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    
    grid = session.findById("wnd[0]/usr/cntlALV0101/shellcont/shell")
    
    cols = ['MATNR',       #товар
            'EMWRK',       #завод
            'ZROUT',       #маршрут
            'DPGCN',       #норматив для подформата
            'DPGPR',       #приоритет в группе замещ.товаров
            'DPGCL',       #обнуление позиции по зам.товара в АЗ
            'TRAME_SO',    #запас в пути с РЦ
            'LABST_SD0',   #запас РЦ
            'TRAME_RI',    #запас в пути на магазин
            'LABST_R_0',   #запас магазина
            'DPGNA',       #число неактивных товаров с запасом
            'DPGWS',       #Число М - неприоритетные товары с запасом
            'MENGE_NE2',   #Заказ, ШТ
            'AQUAN']       #Скорость продаж
    cols_name = ['Тов',    
                 'Звд',
                 'Мрш',
                 'Нрм',       
                 'ПрА',       
                 'Обн_АЗ',
                 'РЦ_путь',
                 'РЦ_з',
                 'Маг_путь',    
                 'Маг_з', 
                 'InAz',
                 'M_АЗ',       
                 'ШТ',   
                 'Скр']
    
    dataframe             = ss.read_alv(grid, cols, cols_name)
    dataframe['Нрм']      = dataframe['Нрм'].astype('int32')
    dataframe['ПрА']      = dataframe['ПрА'].astype('int32')
    dataframe['InAz']     = dataframe['InAz'].astype('int32')
    dataframe['РЦ_путь']  = dataframe['РЦ_путь'].apply(lambda x: ss.replace_minus_int(x))
    dataframe['РЦ_з']     = dataframe['РЦ_з'].apply(lambda x: ss.replace_minus_int(x))
    dataframe['РЦ_з']     = dataframe['РЦ_з'] - dataframe['РЦ_путь']
    dataframe['Маг_путь'] = dataframe['Маг_путь'].apply(lambda x: ss.replace_minus_int(x))
    dataframe['Маг_з']    = dataframe['Маг_з'].apply(lambda x: ss.replace_minus_int(x))
    dataframe['Маг_з']    = dataframe['Маг_з'] + dataframe['Маг_путь']
    dataframe             = dataframe.drop(columns = 'Маг_путь')
    dataframe             = dataframe.drop(columns = 'РЦ_путь')
      
    return dataframe





def count_N(az, inact_stock):
    """Вычисляем N = max(0, норматив - неактив с запасом)"""
    inact_stock_df = inact_stock.reset_index()
    inact_stock_df.columns = ['Звд','Inact']
    azN = az.merge(inact_stock_df)
    azN['N'] = azN['Нрм'] - azN['Inact']
    azN['N'] = azN['N'].apply(lambda x: int(max(0,x)))
    return azN


def count_N_all(az, stock):
    """Вычисляем N = max(0, норматив - неактив с запасом (считаем весь запас вне проекта!))"""
    inact_stock = []
    for plant in plants:
        stock_outproject = stock[(stock['Звд']==plant)&(~stock['Тов'].isin(az[az['Звд']==plant]['Тов']))]
        #Считаем только запас > 0
        plant_inact_stock = len(stock_outproject[stock_outproject['Запас']>0])
        inact_stock.append([plant, plant_inact_stock])
    inact_stock_df = pd.DataFrame(inact_stock, columns = ['Звд','Inact'])
    azN = az.merge(inact_stock_df)
    azN['N'] = azN['Нрм'] - azN['Inact']
    azN['N'] = azN['N'].apply(lambda x: int(max(0,x)))
    
    return azN


def analize_dc(zam, az):
    """Cовмещаем данные из группы замещ.товаров и автозаказа и ставим флаги обнуления
    для цепочки РЦ - магазин"""
    
    final_data = pd.merge(az, zam, on='Тов')
    cols = ['Звд',
            'Мрш',
            'Тов',
            'Пр',
            'ПрА',
            'ЖЦ',
            'Нрм',
            'РЦ_з',
            'Маг_з',
            'Обн1',
            'Обн2',
            'M',
            'M_АЗ',
            'Mf',
            'Обн_ит',
            'Обн_АЗ',
            'ШТ',
            'Скр']
    final_data           = final_data.sort_values(by=['Звд','Пр'])
    final_data['Обн1']   = ''
    final_data['Обн2']   = ''
    final_data['M']      = ''
    final_data['Mf']     = ''
    final_data['Обн_ит'] = ''
    final_data           = final_data[cols]
    
    dataframe = pd.DataFrame()
    
    for plant in plants:
        final_plant_data = final_data[final_data['Звд'] == plant]
        if len(final_plant_data)==0:
            continue    

        #норматив, читаем из АЗ
        N = int(final_plant_data['Нрм'].iloc[0])
    
        #Обнуляем позиции с запасом на РЦ, но не попадающие под норматив
        final_plant_data.loc[final_plant_data.loc[(final_plant_data['РЦ_з'] > 0)].index[N:],'Обн1']='X'
    
        #Считаем M - Количество неприоритетных товаров в группе с запасом на магазине
        #с флагом обнуления, либо с нулевым запасом на РЦ
        M = len(final_plant_data.loc[(final_plant_data['Маг_з'] > 0) & 
                                     ((final_plant_data['РЦ_з'] <= 0) | (final_plant_data['Обн1']=='X'))])
        final_plant_data['M'] = M
        
        #Считаем сколько товаров с ненулевым запасом на РЦ в проекте
        DC_quantity = len(final_plant_data[(final_plant_data['РЦ_з'] > 0)])
        
        #Если с РЦ может поехать меньше товара, чем норматив, то обнулять мы должны меньшее кол-во позиций.
        if DC_quantity < N:
            M = max(0,M - (N - DC_quantity))
        
        final_plant_data['Mf'] = M
            
        mag_quantity = len(final_plant_data[(final_plant_data['Маг_з'] > 0)])
        #Обнуляем последние M товаров среди необнуленных позиций с запасом на РЦ
        if M != 0:
            #если норматив больше позиций, едущих с РЦ, то обнулять имеет смысл только "новые" позиции, 
            #т.е. с нулевым запасом на магазине
            final_plant_data.loc[final_plant_data.loc[(final_plant_data['РЦ_з'] > 0) & 
                                                      (final_plant_data['Обн1']!='X')].index[-M:],['Обн2']]='X'

        #ставим финальный флаг обнуления и дефицита
        final_plant_data.loc[(final_plant_data['Обн1']=='X') | 
                             (final_plant_data['Обн2']=='X'), 'Обн_ит']='X'
        
        final_plant_data.loc[final_plant_data['РЦ_з'] <= 0, 'Обн_ит']='.'
        final_plant_data = final_plant_data.reset_index(drop=True)
        dataframe = dataframe.append(final_plant_data)
    
    return dataframe



def analize_vendor(azN, zam, stock):
    """Cовмещаем данные из группы замещ.товаров и автозаказа и ставим флаги обнуления
    для цепочки РЦ - магазин"""
    
    final_data = pd.merge(azN, zam, on='Тов')
    cols = ['Звд',
            'Мрш',
            'Тов',
            'Пр',
            'ПрА',
            'ЖЦ',
            'Нрм',
            'Маг_з',
            'Обн1',
            'Обн2',
            'N',
            'Inact',
            'InAz',
            'M',
            'M_АЗ',
            #'Mf',
            'Обн_ит',
            'Обн_АЗ',
            'ШТ',
            'Скр']
    final_data           = final_data.sort_values(by=['Звд','Пр'])
    final_data['Обн1']   = ''
    final_data['Обн2']   = ''
    final_data['M']      = ''
    #final_data['Mf']    = ''
    final_data['Обн_ит'] = ''
    final_data           = final_data[cols]
    
    dataframe = pd.DataFrame()
    for plant in plants:
        final_plant_data = final_data[final_data['Звд'] == plant]
        
        plant_pos = len(final_plant_data)   #Количество строк в проекте заказа
        
        if plant_pos==0:
            continue
        N = int(final_plant_data['N'].iloc[0])  #вычислили N раньше для каждого завода
        norm = int(final_plant_data['Нрм'].iloc[0]) #норматив для завода
        
        #Обнуляем позиции, не попавшие в первые N 
        final_plant_data.loc[final_plant_data.iloc[N:].index,'Обн1'] = 'X'

        #Вычисляем M - кол-в товаров с запасом и флагом обнуления
        M = len(final_plant_data.loc[(final_plant_data['Маг_з']>0)&(final_plant_data['Обн1']=='X')])
        
        #Вычисляем M1 - количество товаров с активным ЖЦ на запасах магазина, но не в проектах.
        #Товары на запасах, но не в проектах заказов
        #stock_outproject = stock[(stock['Звд']==plant)&(~stock['Тов'].isin(azN[azN['Звд']==plant]['Тов']))]
        #stock_outproject = stock_outproject.merge(zam) 
        #M1 = len(stock_outproject[(stock_outproject['Запас']>0) & (~stock_outproject['ЖЦ'].isin(inactive_segment))])
        #M1 = len(stock_outproject[(stock_outproject['Запас']>0)])
        
        #Прибавляем M1 к M, увеличивая количество обнуляемых товаров в проекте, 
        #т.к. часть замещающих товаров есть на запасах магазина и больше везти туда не надо
        #M = M + M1
        
        final_plant_data['M'] = M
        
        #Если в проекте может поехать меньше товара, чем норматив, то обнулять мы должны меньшее кол-во позиций.
        #if plant_pos < norm:
        #    M = max(0, M - (norm - plant_pos))
    
        if M != 0:
            final_plant_data.loc[(final_plant_data[final_plant_data['Обн1']!='X'].index[-M:]),['Обн2']]='X'
        final_plant_data.loc[((final_plant_data['Обн1']=='X') | (final_plant_data['Обн2']=='X')), ['Обн_ит']]='X'
        #final_plant_data['Mf'] = M #сохраняем фактическое значение M
        final_plant_data = final_plant_data.reset_index(drop=True)
        
        dataframe = dataframe.append(final_plant_data)
    return dataframe

#-Sub Main-------------------------------------------------------------- 
def Main(method):
    """Расчет по цепочке РЦ - магазин"""
    try:
        SapGuiAuto = win32com.client.GetObject("SAPGUI")
        if not type(SapGuiAuto) == win32com.client.CDispatch:
            return
        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == win32com.client.CDispatch:
            SapGuiAuto = None
            return
        connection = application.Children(0)
        if not type(connection) == win32com.client.CDispatch:
            application = None
            SapGuiAuto = None
            return
        session = connection.Children(0)
        if not type(session) == win32com.client.CDispatch:
            connection = None
            application = None
            SapGuiAuto = None
            return
        
        if method == 'DC':
            zam          = read_zam(session, group_number)          #читаем группу замещающих товаров
            materials    = pd.DataFrame(zam['Тов'])
            stock        = read_stock(session, plants, materials)
            az           = read_az(session, materials)              #автозаказ
            final_data   = analize_dc(zam, az)
            
            return zam, stock, az, final_data
        
        if method == 'Vendor':
            zam          = read_zam(session, group_number)         #читаем группу замещающих товаров
            materials    = pd.DataFrame(zam['Тов'])
            stock        = read_stock(session, plants, materials)  #читаем запасы в ZMB52
            #inact_stock = count_inact_stock(zam, stock)           #считаем запас неактивных товаров
            #zam         = zam.drop(columns='Звд')
            az           = read_az(session, materials)             #читаем автозаказ
            azN          = count_N_all(az, stock)
            final_data   = analize_vendor(azN,zam, stock)
            
            return zam, stock, az, azN, final_data
        
    except:
        print('Ошибка')
        print(sys.exc_info())

    finally:
        session = None
        connection = None
        application = None
        SapGuiAuto = None      
#-Main------------------------------------------------------------------




def total():
    """Выводим весь расчет"""
    for plant in plants:
        final_plant_data = final_data[final_data['Звд']==plant]
        if len(final_plant_data)>0:
            print(final_plant_data, end='\n\n')

            
            
def delta():
    """Выводим строки с отличающимися флагами обнуления"""
    delta = final_data[((final_data['Обн_ит']=='X') & (final_data['Обн_АЗ']!='X'))|((final_data['Обн_ит']!='X') & (final_data['Обн_АЗ']=='X'))]
    for plant in delta['Звд'].unique():
        final_plant_data = final_data[final_data['Звд']==plant]
        if len(final_plant_data)>0:
            print(final_plant_data, end='\n\n')
    
    
    

def test_plant_vendor(test_plant):
    """Выводим детальную информацию по отдельному заводу для поставщик-магазин"""
    final_plant_data = final_data[final_data['Звд']==test_plant]
    #Товары на запасах, но не в проектах заказов
    stock_outproject = stock[(stock['Звд']==test_plant)&(~stock['Тов'].isin(azN[azN['Звд']==test_plant]['Тов']))]
    stock_outproject = stock_outproject.merge(zam)
    #количество товаров с активным ЖЦ на запасах магазина, но не в проектах.
    #stock_outproject[(stock_outproject['Запас']>0) & (~stock_outproject['ЖЦ'].isin(inactive_segment))]
    print('Товары, не вошедшие в проект, но с запасами на магазине:')
    print(stock_outproject[(stock_outproject['Запас']>0)], end='\n\n')
    
    M = final_plant_data['M'].iloc[0]
    norm = final_plant_data['Нрм'].iloc[0]
    plant_pos = len(final_plant_data)
    #Mf = final_plant_data['Mf'].iloc[0]
    #if plant_pos < norm:
    #    print('Расчет фактического М для обнуления (поле Mf):')
    #    print('Mf = max(0, M - (Норматив - позицийПроекта))')
    #    print('Mf = max(0, {0} - ({1} - {2})) = {3}'.format(M,norm,plant_pos,Mf), end='\n\n')
    
    print('Строки проекта заказа:')
    print(final_plant_data)
    
    
    
    
def test_plant_dc(test_plant):
    """Выводим детальную информацию по отдельному заводу для РЦ-магазин"""
    final_plant_data = final_data[final_data['Звд']==test_plant]
    #Товары на запасах, но не в проектах заказов
    stock_outproject = stock[(stock['Звд']==test_plant)&(~stock['Тов'].isin(az[az['Звд']==test_plant]['Тов']))]
    stock_outproject = stock_outproject.merge(zam)
    #количество товаров с активным ЖЦ на запасах магазина, но не в проектах.
    #stock_outproject[(stock_outproject['Запас']>0) & (~stock_outproject['ЖЦ'].isin(inactive_segment))]
    print('Товары, не вошедшие в проект, но с запасами на магазине:')
    if len(stock_outproject) > 0:
        print(stock_outproject[(stock_outproject['Запас']>0)], end='\n\n')
    else:
        print('Таких товаров нет.', end='\n\n')
    

    M = final_plant_data['M'].iloc[0]
    norm = final_plant_data['Нрм'].iloc[0]
    DC_quantity = len(final_plant_data[final_plant_data['РЦ_з'] > 0])
    Mf = final_plant_data['Mf'].iloc[0]
    if DC_quantity < norm:
        print('С РЦ поедет меньше товаров, чем норматив -> обнуляем меньше позиций. \nРасчет фактического М для обнуления (поле Mf):')
        print('Mf = max(0, M - (Норматив - позицийПроекта))')
        print('Mf = max(0, {0} - ({1} - {2})) = {3}'.format(M,norm,DC_quantity,Mf), end='\n\n')
    
    print('Строки проекта заказа:')
    print(final_plant_data)

# Общие константы #

In [3]:
today             = datetime.date.today().strftime("%d.%m.%Y")
company_code      = "1000"    #код компании
inactive_segment  = ['Z1','Z4','Z5']  #неактивные сегменты ЖЦ
variant           = "/script"
variant_zmb52     = "/script"
max_rows          = 30        #максимум строк на экране ALV, выше которого нужно делать page down

## ERD300 ##

In [189]:
group_number = 61      #номер группы замещающих товаров
#plants = ["2010","2011","2013","2014","2015","2016","2017","2018","2019","2020","2059"]        #номер завода
plants = ["2010","2011","2013","2014","2015","2016","2017","2018","2019","2020","2059"]        #номер завода

vendor = '100000000'  #код внешнего поставщика

max_rows = 30          #максимум строк на экране ALV, выше которого нужно делать page down

zam, stock, inact_stock, az, azN, final_data = Main_vendor()
    
for plant in plants:
    final_plant_data = final_data[final_data['Звд']==plant]
    if len(final_plant_data)>0:
        print(final_plant_data, end='\n\n')

Ошибка
(<class 'pywintypes.com_error'>, com_error(-2147352567, 'Ошибка.', (619, 'SAP Frontend Server', 'The control could not be found by id.', 'C:\\Program Files (x86)\\SAP\\FrontEnd\\SAPgui\\sapfront.HLP', 393215, 0), None), <traceback object at 0x000000366DC01908>)


TypeError: 'NoneType' object is not iterable

# ERI #

## РЦ - магазин ##

In [16]:
date              = '01.08.2019'             #дата для чтения проектов заказа
group_number      = 15        #номер группы замещающих товаров
vendor            = ''

#plants = ['2411','3241']
plants = ['2010','2014','2016','2020','2022','2025','2028','2030','2032','2034','2036','2038','2041','2044','2047','2050','2052','2055','2056','2058',
'2060','2062','2064','2066','2068','2070','2076','2078','2082','2084','2089','2091','2093','2095','2098','2100','2102','2104','2107','2109',
'2111','2113','2115','2119','2121','2123','2125','2127','2129','2131','2133','2136','2140','2142','2144','2147','2149','2151','2153','2155',
'2157','2159','2161','2163','2165','2168','2170','2173','2175','2177','2179','2182','2186','2189','2194','2196','2199','2201','2207','2212',
'2214','2220','2223','2225','2411','2413','2415','2417','2419','2421','2423','2426','2428','2430','2432','2436','2438','2440','2442','2444',
'2446','2448','2450','2452','2454','2456','2459','2461','2463','2465','2467','2469','2471','2473','2475','2477','2479','2482','2611','2613',
'2615','2617','2619','2621','2623','2627','2629','2631','2633','2635','2637','2639','2641','2643','2645','2647','2651','2653','2655','2657',
'2659','2661','2663','2665','2667','2669','2671','2673','2676','2678','2680','2682','2684','2686','2688','2690','2692','2696','2698','2700',
'2810','2812','2814','2816','2818','2820','2823','2825','2827','2829','2832','2834','2836','2838','2840','2842','2844','2846','2848','2850',
'2852','2854','2856','2858','2860','2862','2864','2866','2868','2870','2873','2875','2877','2880','2882','2884','2886','2888','2890','2892',
'2894','2896','2898','2900','2904','2906','2908','2910','2912','2914','3011','3013','3015','3018','3020','3022','3024','3026','3028','3030',
'3033','3035','3037','3039','3042','3044','3046','3048','3051','3056','3058','3060','3063','3065','3067','3069','3071','3074','3076','3078',
'3081','3084','3087','3089','3094','3098','3211','3213','3215','3217','3219','3221','3223','3225','3227','3229','3231','3233','3235','3237',
'3239','3242','3245','3247','3249','3251','3253','3255','3257','3259','3262','3264','3266','3268','3270','3272','3274','3276','3278','3411',
'3413','3416','3418','3420','3422','3424','3427','3429','3431','3436','3438','3440','3443','3445','3447','3449','3451','3453','3456','3458',
'3460','3463','3465','3467','3469','3471','3473','3476','3478','3480','3482','3486','3610','3612','3614','3616','3618','3620','3622','3624',
'3626','3628','3630','3632','3635','3638','3640']        #номер завода

zam, stock, az, final_data = Main('DC')

In [20]:
test_plant_dc('2121')

Товары, не вошедшие в проект, но с запасами на магазине:
Таких товаров нет.

С РЦ поедет меньше товаров, чем норматив -> обнуляем меньше позиций. 
Расчет фактического М для обнуления (поле Mf):
Mf = max(0, M - (Норматив - позицийПроекта))
Mf = max(0, 2 - (2 - 1)) = 1

Строки проекта заказа:
    Звд   Мрш         Тов  Пр  ПрА  ЖЦ  Нрм  РЦ_з  Маг_з Обн1 Обн2  M M_АЗ  Mf Обн_ит Обн_АЗ ШТ    Скр
0  2121  1219  1000055305   1    1  Z2    2     0      2            2    2   1      .         0      0
1  2121  1219  1000055306   2    2  Z2    2     0      2            2    2   1      .         0      0
2  2121  1219  1000055307   3    3  Z2    2     9      2         X  2    2   1      X      X  0  0,029


In [17]:
delta()

In [18]:
total()

    Звд   Мрш         Тов  Пр  ПрА  ЖЦ  Нрм  РЦ_з  Маг_з Обн1 Обн2  M M_АЗ  Mf Обн_ит Обн_АЗ ШТ    Скр
0  2010  1219  1000055305   1    1  Z2    2     0      2            2    2   1      .         0  0,029
1  2010  1219  1000055306   2    2  Z2    2     0      2            2    2   1      .         0  0,033
2  2010  1219  1000055307   3    3  Z2    2     9      2         X  2    2   1      X      X  0  0,032

    Звд   Мрш         Тов  Пр  ПрА  ЖЦ  Нрм  РЦ_з  Маг_з Обн1 Обн2  M M_АЗ  Mf Обн_ит Обн_АЗ ШТ Скр
0  2014  1219  1000055305   1    1  Z2    2     0      2            2    2   0      .         0   0
1  2014  1219  1000055306   2    2  Z2    2     0      1            2    2   0      .         0   0

    Звд   Мрш         Тов  Пр  ПрА  ЖЦ  Нрм  РЦ_з  Маг_з Обн1 Обн2  M M_АЗ  Mf Обн_ит Обн_АЗ ШТ Скр
0  2016  1219  1000055305   1    1  Z2    2     0      3            2    2   0      .         0   0
1  2016  1219  1000055306   2    2  Z2    2     0      1            2    2   0      . 

## Поставщик - магазин ##

In [24]:
date             = today             #дата для чтения проектов заказа
group_number     = 16                #номер группы замещающих товаров
vendor           = '175002315'       #код внешнего поставщика

#plants = ['2010','2016','2022','2038','2076','2095']
plants = ['2010','2014','2016','2020','2022','2025','2028','2030','2032','2034','2036','2038','2041','2044','2047','2050','2052','2055','2056','2058',
'2060','2062','2064','2066','2068','2070','2076','2078','2082','2084','2089','2091','2093','2095','2098','2100','2102','2104','2107','2109',
'2111','2113','2115','2119','2121','2123','2125','2127','2129','2131','2133','2136','2140','2142','2144','2147','2149','2151','2153','2155',
'2157','2159','2161','2163','2165','2168','2170','2173','2175','2177','2179','2182','2186','2189','2194','2196','2199','2201','2207','2212',
'2214','2220','2223','2225','2411','2413','2415','2417','2419','2421','2423','2426','2428','2430','2432','2436','2438','2440','2442','2444',
'2446','2448','2450','2452','2454','2456','2459','2461','2463','2465','2467','2469','2471','2473','2475','2477','2479','2482','2611','2613',
'2615','2617','2619','2621','2623','2627','2629','2631','2633','2635','2637','2639','2641','2643','2645','2647','2651','2653','2655','2657',
'2659','2661','2663','2665','2667','2669','2671','2673','2676','2678','2680','2682','2684','2686','2688','2690','2692','2696','2698','2700',
'2810','2812','2814','2816','2818','2820','2823','2825','2827','2829','2832','2834','2836','2838','2840','2842','2844','2846','2848','2850',
'2852','2854','2856','2858','2860','2862','2864','2866','2868','2870','2873','2875','2877','2880','2882','2884','2886','2888','2890','2892',
'2894','2896','2898','2900','2904','2906','2908','2910','2912','2914','3011','3013','3015','3018','3020','3022','3024','3026','3028','3030',
'3033','3035','3037','3039','3042','3044','3046','3048','3051','3056','3058','3060','3063','3065','3067','3069','3071','3074','3076','3078',
'3081','3084','3087','3089','3094','3098','3211','3213','3215','3217','3219','3221','3223','3225','3227','3229','3231','3233','3235','3237',
'3239','3242','3245','3247','3249','3251','3253','3255','3257','3259','3262','3264','3266','3268','3270','3272','3274','3276','3278','3411',
'3413','3416','3418','3420','3422','3424','3427','3429','3431','3436','3438','3440','3443','3445','3447','3449','3451','3453','3456','3458',
'3460','3463','3465','3467','3469','3471','3473','3476','3478','3480','3482','3486','3610','3612','3614','3616','3618','3620','3622','3624',
'3626','3628','3630','3632','3635','3638','3640']

zam, stock, az, azN, final_data = Main('Vendor')

In [27]:
test_plant_vendor('2055')

Товары, не вошедшие в проект, но с запасами на магазине:
    Звд         Тов  Запас  Пр  ЖЦ
0  2055  1000019144      1   8  Z4
1  2055  1000019145      1   0  Z4
2  2055  1000019146      1  10  Z4
3  2055  1000034780      2  12  Z4
4  2055  1000051134      1  14  Z4

Строки проекта заказа:
    Звд   Мрш         Тов  Пр  ПрА  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  InAz  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2055  1378  1000087445   1    1  Z3    3      1    X       0      5     4  6    7      X      X  0      0
1  2055  1378  1000087444   2    2  Z3    3      1    X       0      5     4  6    7      X      X  0      0
2  2055  1378  1000087447   3    3  Z3    3      1    X       0      5     4  6    7      X      X  0      0
3  2055  1378  1000087446   4    4  Z3    3      1    X       0      5     4  6    7      X      X  0      0
4  2055  1378  1000087443   5    5  Z3    3      2    X       0      5     4  6    7      X      X  0  0,031
5  2055  1378  1000087442   6    6  Z3    3      1    X

In [25]:
delta()

In [26]:
total()

    Звд   Мрш         Тов  Пр  ПрА  ЖЦ  Нрм  Маг_з Обн1 Обн2  N  Inact  InAz  M M_АЗ Обн_ит Обн_АЗ ШТ    Скр
0  2055  1378  1000087445   1    1  Z3    3      1    X       0      5     4  6    7      X      X  0      0
1  2055  1378  1000087444   2    2  Z3    3      1    X       0      5     4  6    7      X      X  0      0
2  2055  1378  1000087447   3    3  Z3    3      1    X       0      5     4  6    7      X      X  0      0
3  2055  1378  1000087446   4    4  Z3    3      1    X       0      5     4  6    7      X      X  0      0
4  2055  1378  1000087443   5    5  Z3    3      2    X       0      5     4  6    7      X      X  0  0,031
5  2055  1378  1000087442   6    6  Z3    3      1    X       0      5     4  6    7      X      X  0      0



3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]


# ERP
## РЦ - магазин ##

In [4]:
date              = '31.10.2019'             #дата для чтения проектов заказа
group_number      = 71        #номер группы замещающих товаров
vendor            = ''

plants = ['2663','2645']
#plants = ['3213','3215','3217','3219','3221','3223','3225','3227','3229','3231','3233','3235','3237',
#'3239','3242','3245','3247','3249','3251','3253','3255','3257','3259','3262','3264','3266',
#          '3268','3270','3272','3274','3276','3278','3411',
#'3413','3416','3418','3420','3422','3424','3427','3429','3431','3436']        #номер завода

zam, stock, az, final_data = Main('DC')

In [8]:
test_plant_dc('2663')

Товары, не вошедшие в проект, но с запасами на магазине:
Таких товаров нет.

Строки проекта заказа:
    Звд   Мрш         Тов  Пр  ПрА  ЖЦ  Нрм  РЦ_з  Маг_з Обн1 Обн2  M M_АЗ  Mf Обн_ит Обн_АЗ ШТ    Скр
0  2663  1455  1000035642   1    1  Z4    1   312     13            0    0   0                0  0,119
1  2663  1455  1000091720   2    2  Z2    1     0      0            0    0   0      .         0      0


In [6]:
delta()